本次作业以垃圾邮件分类任务为基础，要求提取文本特征并使用朴素贝叶斯算法进行垃圾邮件识别（调用已有工具包或自行实现）。

### 任务介绍
电子邮件是互联网的一项重要服务，在大家的学习、工作和生活中会广泛使用。但是大家的邮箱常常被各种各样的垃圾邮件填充了。有统计显示，每天互联网上产生的垃圾邮件有几百亿近千亿的量级。因此，对电子邮件服务提供商来说，垃圾邮件过滤是一项重要功能。而朴素贝叶斯算法在垃圾邮件识别任务上一直表现非常好，至今仍然有很多系统在使用朴素贝叶斯算法作为基本的垃圾邮件识别算法。

本次实验数据集来自[Trec06](https://plg.uwaterloo.ca/cgi-bin/cgiwrap/gvcormac/foo06)的中文垃圾邮件数据集，目录解压后包含三个文件夹，其中data目录下是所有的邮件（未分词），已分词好的邮件在data_cut目录下。邮件分为邮件头部分和正文部分，两部分之间一般有空行隔开。标签数据在label文件夹下，文件中每行是标签和对应的邮件路径。‘spam’表示垃圾邮件，‘ham’表示正常邮件。

本次实验

基本要求：
1. 提取正文部分的文本特征；
2. 划分训练集和测试集（可以借助工具包。一般笔记本就足够运行所有数据，认为实现困难或算力不够的同学可以采样一部分数据进行实验。）；
3. 使用朴素贝叶斯算法完成垃圾邮件的分类与预测，要求测试集准确率Accuracy、精准率Precision、召回率Recall均高于0.9（本次实验可以使用已有的一些工具包完成如sklearn）；
4. 对比特征数目（词表大小）对模型效果的影响；
5. 提交代码和实验报告。

扩展要求：
1. 邮件头信息有时也可以协助判断垃圾邮件，欢迎学有余力的同学们尝试；
2. 尝试自行实现朴素贝叶斯算法细节；
3. 尝试对比不同的概率计算方法。

### 导入工具包

In [53]:
'''
提示：
若调用已有工具包，sklearn中提供了一些可能会用到的类。
'''
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # 提取文本特征向量的类
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, ComplementNB # 三种朴素贝叶斯算法，差别在于估计p(x|y)的方式
from sklearn.model_selection import train_test_split # 分割数据集

In [54]:
import pandas as pd
# 读取标签索引
# 读取label文件
indexData = pd.read_csv('trec06c-utf8/label/index', sep=' ', header=None, names=['label', 'path'])
indexData.path = indexData.path.map(lambda x: x.replace('../data', './trec06c-utf8/data_cut'))
# indexData
indexData['text'] = indexData.path.map(lambda x: open(x, 'r', encoding='utf-8').read())

In [55]:
indexData['content'] = indexData.text.map(lambda x: x.split('\n\n',1)[1])
indexData.head()


,label,path,text,content
0,spam,./trec06c-utf8/data_cut/000/000,Received: from hp-5e1fe6310264 ([218.79.188.13...,[ 课 程 背 景 ]\n\n \n每 一位 管理 和 技术人员 都 清楚 地 ...
1,ham,./trec06c-utf8/data_cut/000/001,Received: from jdl.ac.cn ([159.226.42.8])\n\tb...,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ， 跟 儿子 感情 不 和...
2,spam,./trec06c-utf8/data_cut/000/002,Received: from 163.con ([61.141.165.252])\n\tb...,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！\n我 是 深圳 金海 实业 有...
3,spam,./trec06c-utf8/data_cut/000/003,Received: from 12.com ([222.50.6.150])\n\tby s...,贵 公司 负责人 ( 经理 / 财务 ） 您好 ：\n深圳市 华龙 公司 受 多家 公司 委...
4,spam,./trec06c-utf8/data_cut/000/004,Received: from dghhkjk.com ([59.36.183.208])\n...,这是 一封 HTML 格式 信件 ！\n\n- - - - - - - - - - - - ...


In [56]:
# 分割数据集
# 特征提取
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(indexData['content'])
# print(X)
Y = indexData['label']
train_data, test_data, train_label, test_label = train_test_split(X, Y, test_size=0.2, random_state=42)
print(train_data.shape, test_data.shape, train_label.shape, test_label.shape)

(51696, 196902) (12924, 196902) (51696,) (12924,)


In [57]:
clf = MultinomialNB()
clf.fit(train_data, train_label)
# 预测
y_pred = clf.predict(test_data)
from sklearn.metrics import accuracy_score, precision_score, recall_score
# accuracy
accuracy = accuracy_score(test_label, y_pred)
print('accuracy: ', accuracy)
# precision
precision = precision_score(test_label, y_pred, pos_label='spam')
print('precision: ', precision)
# recall
recall = recall_score(test_label, y_pred, pos_label='spam')
print('recall: ', recall)

accuracy:  0.9800371402042711
precision:  0.9800573065902579
recall:  0.9902721482339317


In [58]:
# 